# Proyecto 2 Ironhack. 
Analisis de Lista de Reproduccion. Generacion de una lista de reproduccion nueva

In [1]:
import base64
import requests
import datetime
from urllib.parse import urlencode
import pandas as pd

## 1. Coneccion con la API de Spotify

In [2]:
# Llaves de acceso para conectarnos a la API de Spotify
client_id = '8e5f91c2fc27453eb255a039b42bcb70'
client_secret = '09645aff5c8849f2baab977e4cfd1017'
user_id = '1279741536'

In [32]:
# Generamos esta classe que nos funcionará para que de manera automatica se genere toda la solicitud de acceso a la API

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    client_id = None
    client_secret = None
    access_token_did_expire = True
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    # Aqui se crean las credenciales en base 64 para poder acceder a la API
    def get_client_credentials(self):
        """ returns a base64 encoded string """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())        
        return client_creds_b64.decode()
    
    # Aqui estamos generando los headers de acceso
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        
        return {
            "Authorization":f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials"
        }
    
    # Aqui nos autentificamos y obtenemos el token de acceso
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data = token_data, headers = token_headers)

        if r.status_code not in range(200,299):
            raise Exceptiontion('Could not authenticate client')
        
            
        data = r.json()
        now = datetime.datetime.now()
        acces_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = acces_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    # Extraemos el token de acceso
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    # Aqui son los tokens de para las busquedas
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
        
    # busqueda de listas de reproduccion
    def get_user_playlist(self,u_id):
        endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las canciones por playlist
    def get_song_playlist(self,playlist_id):
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las canciones por playlist
    def get_audio_features(self,song_id):
        endpoint = f'https://api.spotify.com/v1/audio-features/{song_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [33]:
# Llamamos a la funcion y le asignamos nuestros client ID y Client Secret
spotify = SpotifyAPI(client_id,client_secret)

In [34]:
# Nos indica si fue realizada la coneccion con la API
spotify.perform_auth()

True

In [35]:
# Obtenemos el access_token que nos servirá para las consultas nescesarias. Por eso lo guardamos en una variable y que sea mas
# sencillo llamarlo posteriormente.
access_token = spotify.access_token
access_token

'BQDQ-7YxinCG5kS11Xe7-eIM_YrYJab0wCHxYckjepuM4zIy-n4OA4D-FTPcntm3GJgQ1l7lOiuRq8jsBmg'

## 2. Busqueda y descarga de la informacion de las listas de reproduccion

In [36]:
# Obtenemos las listas de reproduccion en mi usuario
playlist_rikrdinii = spotify.get_user_playlist(user_id)
columns = ['name','id','tracks']
pl_rikrdinii = pd.DataFrame(playlist_rikrdinii['items'],columns=columns)
pl_rikrdinii.head()


,name,id,tracks
0,Mis pistas en Shazam,4cneXKJlVKFLKAYuwtk57I,{'href': 'https://api.spotify.com/v1/playlists...
1,Dubstep 2020​​ ‍,5wNRJwSnBImUuZXJW1TiAj,{'href': 'https://api.spotify.com/v1/playlists...
2,Pure Rock & Roll,37i9dQZF1DWWRktbhJiuqL,{'href': 'https://api.spotify.com/v1/playlists...
3,Fiesta Indie,37i9dQZF1DWSXAHZTiX76o,{'href': 'https://api.spotify.com/v1/playlists...
4,Radio Rock,37i9dQZF1DX1te6miphixI,{'href': 'https://api.spotify.com/v1/playlists...


In [46]:
# Generamos una lista con los ID de cada Lista de reproduccion
id_playlist = [playlist_rikrdinii['items'][i]['id'] for i in range(len(playlist_rikrdinii['items']))]
id_playlist[1]

'5wNRJwSnBImUuZXJW1TiAj'

In [42]:
# Analizamos una lista de reproduccion mediante su ID
lista_1 = spotify.get_song_playlist('7M3zmWRA291rxcI1vw47DT')

# Obtenemos la info de las canciones en la lista de reproduccion
canciones = []
for a in id_playlist:
    lista_1 = spotify.get_song_playlist(a)
    for i in range(len(lista_1['items'])):
        id_cancion = lista_1['items'][i]['track']['id']
        nombre_cancion = lista_1['items'][i]['track']['name']
        album =lista_1['items'][i]['track']['album']['name']
        artista = lista_1['items'][i]['track']['artists'][0]['name']

        canciones.append([id_cancion,nombre_cancion,album,artista])

canciones

[['7gHs73wELdeycvS48JfIos', 'Faded', 'Faded', 'Alan Walker'],
 ['5khPVYWIdAE0kSTqekYIwB',
  'Blah Blah Blah',
  'Blah Blah Blah EP',
  'Armin van Buuren'],
 ['3JOVTQ5h8HGFnDdp4VT3MP',
  'Mad World (Feat. Michael Andrews)',
  'Trading Snakeoil for Wolftickets',
  'Gary Jules'],
 ['3JOVTQ5h8HGFnDdp4VT3MP',
  'Mad World (Feat. Michael Andrews)',
  'Trading Snakeoil for Wolftickets',
  'Gary Jules'],
 ['1Ob57NqUydp5cC3FCNV4Gy',
  'I Feel You - 2006 Remaster',
  'Songs of Faith and Devotion (2006 Remaster)',
  'Depeche Mode'],
 ['0xCdQ9jYFDENkE2ck73CAc',
  'Save Yourself - NGHTMRE VIP REMIX',
  'Save Yourself (NGHTMRE VIP REMIX)',
  'The Chainsmokers'],
 ['3eY4uiXIJQFy5xuOpYRylW',
  'Ignite (feat. SEUNGRI) - Panta.Q Remix',
  'Ignite (feat. SEUNGRI) [Remixes]',
  'K-391'],
 ['1zhQTZf9Xd9Kjla457Bxiu',
  'Too Far Gone - Rob Gasser Remix',
  'Too Far Gone (Remixes) (feat. Anna Clendening)',
  'Lost Kings'],
 ['1JcC5Zu7vrdMGDjJkw6Z3Q',
  'Faded - Slushii Remix',
  'Faded (Remixes II)',
  'Alan 

In [43]:
# ID de las canciones
for i in canciones:
    print(i[0])

7gHs73wELdeycvS48JfIos
5khPVYWIdAE0kSTqekYIwB
3JOVTQ5h8HGFnDdp4VT3MP
3JOVTQ5h8HGFnDdp4VT3MP
1Ob57NqUydp5cC3FCNV4Gy
0xCdQ9jYFDENkE2ck73CAc
3eY4uiXIJQFy5xuOpYRylW
1zhQTZf9Xd9Kjla457Bxiu
1JcC5Zu7vrdMGDjJkw6Z3Q
5xZjHkJ5TuHNMJeraaOaJC
1b1TQHFL6lSFh8hb1mgiHs
21ahVFLZPeDQmOiIItPqkx
7Bvp5JUJiyUrMgvH4DR0wH
0IrEswfPqkHn7iGUoCDPAb
4NgJNCpvxBIQ3stPdF9epC
24hYr62Cz2iA3OOef8mykI
7CJEFjKOk5CEAN9C248DaM
76I3PmbGZazzNlEwlp1y85
1zZ8DfrLf4fQgARxIKOs20
16i3H5whsxhyLCBwceuR4j
23waejgtiOrlz2t0xO7Zsd
5wsZ9fE3f06lrbYeS6A2Oq
74eK1Efmvt8j6Qyf44p8JN
4hJolpt4ZqzoZuET5l3rzV
5LfkJez2ixlKfbQCZ0UU65
2jdGOeIjQNuPXx8rml1FkX
4TBkWyEx4FCuyxLtMm5r3b
3VvV7C2OFCIBpQTBD9eBQP
3ngFMSKXhfhIuIF4CueOsr
1jiczAUjgZ9MF6TsYJFAxx
6lwxNGjvD1pFlXyNI2h9aH
4iPmzKKyYrxRBTy0jOBaNh
5SVWUrkhwUkdSJXa6qdrDg
7GvYheSEpubw6JyUzZWxtD
2f3dQn9jfUwkgnPreXgByt
1eVTKOOPGCRjzwslb6AzZx
6JFlP9L1BfNiS5V4akpoLh
6Ox3MZK1pRPF1fF8s4Y5bA
4ixAVZZD7evMMZUQpQcUSS
77r1W7Xf5CXtWcwhGMA8ZV
2FqbTIMNO86qGrRGTIfmtG
0zl9JGNm3Ob7XyMO0jqRwz
0LnG7EqyO9XkSdsOeRAUDB
5q1NLG5tEns

In [54]:
# Obtener las caracteristicas de audio
audio = spotify.get_audio_features('2U10qQg2DY80o590FcA4gx')
df = pd.DataFrame(audio, ['id'])
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
id,0.526,0.881,10,-4.92,1,0.0454,0.0535,0.00667,0.972,0.31,110.402,audio_features,2U10qQg2DY80o590FcA4gx,spotify:track:2U10qQg2DY80o590FcA4gx,https://api.spotify.com/v1/tracks/2U10qQg2DY80...,https://api.spotify.com/v1/audio-analysis/2U10...,273800,4


In [51]:
for i in canciones:
    df_audiofeatures = pd.DataFrame(spotify.get_audio_features(i[0]),index=['id'])
    
df_audiofeatures

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
id,0.557,0.323,2,-7.773,1,0.106,0.838,0.000018,0.301,0.0791,78.862,audio_features,3b4749nnFuPGUJiYtuLhT2,spotify:track:3b4749nnFuPGUJiYtuLhT2,https://api.spotify.com/v1/tracks/3b4749nnFuPG...,https://api.spotify.com/v1/audio-analysis/3b47...,251787,4


In [11]:
# Documentacion para las Playlist: 
# https://developer.spotify.com/documentation/web-api/reference/playlists/get-a-list-of-current-users-playlists/

# Maximo se pueden obtener 50 listas de reproduccion
# https://api.spotify.com/v1/me/playlists" -H "Authorization: Bearer {your access token} --> Ejemplo para extraer las listas
# Tres Llamadas Get para descargar la data 

In [12]:
# Documentacion Obtencion de info de las listas de reproduccion:
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/

## 3. Analisis de las listas de reproduccion

## 4. Generacion de las nuevas listas de reproduccion con canciones que tengan caracteristicas similares a las de las listas de reproduccion generadas.

In [13]:
# un metodo post para subir la lista de reproduccion

## 5. BONUS: Scrapear lista de reproduccion de wikipedia de Luke Cage y generar la lista de reproduccion pertinente